# Python for .NET

Python for .NET (`pythonnet`, `Python.NET`) is a package that gives Python programmers
nearly seamless integration with the .NET 4.0+ Common Language Runtime
(CLR) on Windows and Mono runtime on Linux and OSX. Python for .NET
provides a powerful application scripting tool for .NET developers.
Using this package you can script .NET applications or build entire
applications in Python, using .NET services and components written in
any language that targets the CLR (C#, VB.NET, F#, C++/CLI).

Note that this package does _not_ implement Python as a first-class CLR
language - it does not produce managed code (IL) from Python code. Rather,
it is an integration of the CPython engine with the .NET or Mono runtime.
This approach allows you to use CLR services and continue to use existing
Python code and C-API extensions while maintaining native execution
speeds for Python code. If you are interested in a pure managed-code
implementation of the Python language, you should check out the
[IronPython][1] project, which is in active development.

Python for .NET is currently compatible and tested with Python releases
`2.7`, `3.3`, `3.4`, `3.5`, and `3.6`.
Current releases are available at the [Python for .NET website][2].
To subscribe to the [Python for .NET mailing list][3] or read the
[online archives][4] of the list, see the [mailing list information][3]
page. Use the [Python for .NET issue tracker][55] to report issues.

-   The document provides a detailed overview of Python for .NET,
    as well as some basic usage examples. Many other examples can be
    found in the demos and unit tests for the package.
-   Checkout the [`Python.NET`][77] code from github.
-   [Download releases][6] for various versions of Python and CLR.

[1]: http://www.ironpython.com

[2]: http://pythonnet.github.io/

[3]: http://mail.python.org/mailman/listinfo/pythondotnet

[4]: http://mail.python.org/pipermail/pythondotnet/

[5]: https://github.com/pythonnet/pythonnet/releases

[6]: https://pypi.python.org/pypi/pythonnet

[7]: http://www.go-mono.com

[8]: https://github.com/pythonnet/pythonnet/blob/master/README.md

[9]: http://pythonnet.github.io/LICENSE

[10]: http://www.python.org/license.html

[55]: http://github.com/pythonnet/pythonnet/issues

[66]: http://pythonnet.github.io/readme.html

[77]: http://github.com/pythonnet/pythonnet

## Installation

Python for .NET is available as a source release on [GitHub][5] and as a
binary wheel distribution for all supported versions of Python and the
common language runtime from the [Python Package Index][6].

The source release is a self-contained "private" assembly. Just unzip the
package wherever you want it, cd to that directory, build the solution
`python setup.py build_ext --inplace`. Once you start up Python or
IPython interpreter in this directory or append this directory to
`sys.path`, then after `import clr` statement .NET assemblies can
be used.
You can also run `nPython.exe` (`mono nPython.exe` on `*nix`) to check
how python can be embedded in console .NET application. Note that the
source release does not include a copy of the CPython runtime, so you will
need to have installed Python on your machine before using the source release.

**Running on Linux/Mono:** Unit testing shows that `Python.NET` will run
under [Mono][7], though the Mono runtime is less supported so there still
may be problems.



[5]: https://github.com/pythonnet/pythonnet/releases

[6]: https://pypi.python.org/pypi/pythonnet

[7]: http://www.go-mono.com


## Getting Started

A key goal for this project has been that Python for .NET should
"work just the way you'd expect in Python", except for cases that are
.NET specific (in which case the goal is to work "just the way
you'd expect in C#"). In addition, with the IronPython project having
established a community, it is my goal that code written for IronPython
run without modification under Python for .NET.

If you already know Python, you can probably finish this readme and then
refer to .NET docs to figure out anything you need to do. Conversely if
you are familiar with C# or another .NET language, you probably just need
to pick up one of the many good Python books or read the Python tutorial
online to get started.

A good way to start is to interactively explore .NET usage in python
interpreter by following along with the examples in this document. If you
get stuck, there are also a number of demos and unit tests located in the
source directory of the distribution that can be helpful as examples.

## Importing Modules

Python for .NET allows CLR namespaces to be treated essentially as
Python packages.

In [1]:
import clr
from System import String
clr.AddReference('System.Collections')
from System.Collections import *

Types from any loaded assembly may be imported and used in this manner.
To load an assembly, use the `AddReference` function in the `clr` module:

In [3]:
clr.AddReference("System.Windows.Forms")
from System.Windows.Forms import Form

**Note** Earlier releases of Python for .NET relied on "implicit loading"
to support automatic loading of assemblies whose names corresponded to an
imported namespace. Implicit loading still works for backward compatibility,
but will be removed in a future release so it is recommended to use
the `clr.AddReference` method.

Python for .NET uses the PYTHONPATH (sys.path) to look for assemblies
to load, in addition to the usual application base and the GAC. To ensure
that you can implicitly import an assembly, put the directory containing
the assembly in `sys.path`.

## Using Classes

Python for .NET allows you to use any non-private classes, structs,
interfaces, enums or delegates from Python. To create an instance of a
managed class, you use the standard instantiation syntax, passing a set
of arguments that match one of its public constructors:

In [4]:
from System.Drawing import Point

p = Point(5, 5)

In most cases, Python for .NET can determine the correct constructor to
call automatically based on the arguments. In some cases, it may be necessary
to call a particular overloaded constructor, which is supported by a special
`__overloads__` attribute, which will soon be deprecated in favor of
iPy compatible "Overloads", on a class:

In [6]:
from System import String, Char, Int32

s = String.Overloads[Char, Int32]('A', 10)
s = String.Overloads[Char, Int32]('A', 10)

## Using Generics

`Python.NET` also supports generic types. A generic type must be bound to
create a concrete type before it can be instantiated. Generic types support
the subscript syntax to create bound types:

In [7]:
from System.Collections.Generic import Dictionary
from System import *

dict1 = Dictionary[String, String]()
dict2 = Dictionary[String, Int32]()
dict3 = Dictionary[String, Type]()

When you pass a list of types using the subscript syntax, you can also
pass a subset of Python types that directly correspond to .NET types:

In [8]:
dict1 = Dictionary[str, str]()
dict2 = Dictionary[str, int]()
dict3 = Dictionary[str, Decimal]()

This shorthand also works when explicitly selecting generic methods or
specific versions of overloaded methods and constructors (explained later).

You can also subclass managed classes in Python, though members of the
Python subclass are not visible to .NET code. See the `helloform.py` file
in the `/demo` directory of the distribution for a simple Windows Forms
example that demonstrates subclassing a managed class.

## Fields And Properties

You can get and set fields and properties of CLR objects just as if they
were regular attributes:

In [9]:
from System import Environment

name = Environment.MachineName
Environment.ExitCode = 1

## Using Indexers

If a managed object implements one or more indexers, you can call the
indexer using standard Python indexing syntax:

In [10]:
from System.Collections import Hashtable

table = Hashtable()
table["key 1"] = "value 1"

Overloaded indexers are supported, using the same notation one would
use in C#:

In [14]:
#Loading clrmagic extension (pip install clrmagic) to generate runtime C# code, usable from IPython
%reload_ext clrmagic

Below is runtime generated method `idxnames` that shows example of overloading indexer in CLR type `IndexedNames`

In [36]:
%%CS idxnames System.dll
static IndexedNames idxn;
public static object idxnames()
{
    if (idxn == null) idxn = new IndexedNames();
    return idxn;
}
class IndexedNames
{
  private string[] namelist = new string[size];
  static public int size = 10;
  public IndexedNames()
  {
     for (int i = 0; i < size; i++)
     {
        namelist[i] = "N. A.";
     }
  }

  public string this[int index]
  {
     get
     {
        string tmp;

        if( index >= 0 && index <= size-1 )
        {
           tmp = namelist[index];
        }
        else
        {
           tmp = "";
        }

        return ( tmp );
     }
     set
     {
        if( index >= 0 && index <= size-1 )
        {
           namelist[index] = value;
        }
     }
  }
  public int this[string name]
  {
     get
     {
        int index = 0;
        while(index < size)
        {
           if (namelist[index] == name)
           {
            return index;
           }
           index++;
        }
        return index;
     }

  }
}

<function clrmagic.create_cs_function.<locals>.<lambda>>

In [44]:
idx1=idxnames()

idx1[5]="abc"

idx1["abc"], idx1[5], idx1[0]

(5, 'abc', 'N. A.')

## Using Methods

Methods of CLR objects behave generally like normal Python methods.
Static methods may be called either through the class or through an
instance of the class. All public and protected methods of CLR objects
are accessible to Python:

In [47]:
from System import Environment

drives = Environment.GetLogicalDrives()
list(drives)

['C:\\', 'D:\\']

It is also possible to call managed methods `unbound`
(passing the instance as the first argument) just as with Python methods.
This is most often used to explicitly call methods of a base class.

**Note** There is one caveat related to calling unbound methods: it is
possible for a managed class to declare a static method and an instance
method with the same name. Since it is not possible for the runtime to know
the intent when such a method is called unbound, the static method will
always be called.

The docstring of CLR a method (**doc**) can be used to view the signature
of the method, including overloads if the CLR method is overloaded.
You can also use the Python `help` method to inspect a managed class:

In [50]:
from System import Environment

print(Environment.GetFolderPath.__doc__)

help(Environment)

System.String GetFolderPath(SpecialFolder)
System.String GetFolderPath(SpecialFolder, SpecialFolderOption)
Help on class Environment in module System:

class Environment(Object)
 |  Void .ctor()
 |  
 |  Method resolution order:
 |      Environment
 |      Object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __call__(self, /, *args, **kwargs)
 |      Call self as a function.
 |  
 |  __delitem__(self, key, /)
 |      Delete self[key].
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getitem__(self, key, /)
 |      Return self[key].
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __hash__(self, /)
 |      Return hash(self).
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __le__(self, value, /)
 |      Return self<=value.
 |  
 |  __lt__(s

## Overloaded and Generic Methods

While Python for .NET will generally be able to figure out the right
version of an overloaded method to call automatically, there are cases
where it is desirable to select a particular method overload explicitly.

Methods of CLR objects have an `__overloads__`, which will soon be
deprecated in favor of iPy compatible Overloads, attribute that can be
used for this purpose:

In [54]:
from System import Console

# in jupyter writes to nbserver console
Console.WriteLine.Overloads[bool](True) 
Console.WriteLine.Overloads[str]("true")
Console.WriteLine.Overloads[int](42)

Similarly, generic methods may be bound at runtime using the subscript
syntax directly on the method:

In [63]:
%%CS GenericMethods System.dll
static TypeWithGenerics twg;

public static object GenericMethods()
{
    if (twg == null) twg = new TypeWithGenerics();
    return twg;
}

class TypeWithGenerics
{
  public TypeWithGenerics()
  {
      
  }

  public System.Type genericmethod<T>(T g)
  {
     return g.GetType();
  }
}

<function clrmagic.create_cs_function.<locals>.<lambda>>

In [69]:
twg = GenericMethods()

twg.genericmethod[int](10).Name, twg.genericmethod[str]("10").Name

('Int32', 'String')

## Delegates And Events

Delegates defined in managed code can be implemented in Python.
A delegate type can be instantiated and passed a callable Python object
to get a delegate instance. The resulting delegate instance is a true
managed delegate that will invoke the given Python callable when it
is called:

In [78]:
def my_handler(source, args):
    print('my_handler called!')

# instantiate a delegate
d = AssemblyLoadEventHandler(my_handler)

# use it as an event handler
AppDomain.CurrentDomain.AssemblyLoad += d


Multicast delegates can be implemented by adding more callable objects to
a delegate instance:

```python
class test(object):
    def method1():
        return "method1"
    def method2():
        return "method2"
t=test()
d += t.method1
d += t.method2
d()
```

Events are treated as first-class objects in Python, and behave in many
ways like methods. Python callbacks can be registered with event attributes,
and an event can be called to fire the event.

Note that events support a convenience spelling similar to that used in C#.
You do not need to pass an explicitly instantiated delegate instance to an
event (though you can if you want). Events support the `+=` and `-=`
operators in a way very similar to the C# idiom:

```python
def handler(source, args):
    print('my_handler called!')

# register event handler
object.SomeEvent += handler

# unregister event handler
object.SomeEvent -= handler

# fire the event
result = object.SomeEvent(...)
```

## Exception Handling

You can raise and catch managed exceptions just the same as you would
pure-Python exceptions:

In [80]:
from System import NullReferenceException

try:
    raise NullReferenceException("aiieee!")
except NullReferenceException as e:
    print(e.Message)
    print(e.Source)


aiieee!
None


## Using Arrays

The type `System.Array` supports the subscript syntax in order to
make it easy to create managed arrays from Python:

In [88]:
from System import Array

#This is regression - TypeError: Cannot convert 10 to System.Int32[]
#myarray = Array[int](10)

myarray = Array[int]([1,2])
list(myarray)

[1, 2]

Managed arrays support the standard Python sequence protocols:

In [92]:
items = Array[int]([1,2])

# Get first item
v = items[0]
items[0] = v
print(v)

# Get last item
v = items[-1]
items[-1] = v
print(v)

# Get length
l = len(items)
print(l)

# Containment test
test = v in items
print(test)

1
2
2
True


Multidimensional arrays support indexing using the same notation one
would use in C#:

```python
items[0, 2]

items[0, 2, 3]
```

## Using Collections

Managed arrays and managed objects that implement the IEnumerable
interface can be iterated over using the standard iteration
Python idioms:

In [95]:
import System
domain = System.AppDomain.CurrentDomain

for item in domain.GetAssemblies():
    name = item.GetName()
    print(name)

mscorlib, Version=4.0.0.0, Culture=neutral, PublicKeyToken=b77a5c561934e089
clrmodule, Version=2.4.0.0, Culture=neutral, PublicKeyToken=null
Python.Runtime, Version=2.4.0.0, Culture=neutral, PublicKeyToken=null
System.Core, Version=4.0.0.0, Culture=neutral, PublicKeyToken=b77a5c561934e089
System, Version=4.0.0.0, Culture=neutral, PublicKeyToken=b77a5c561934e089
__CodeGenerator_Assembly, Version=0.0.0.0, Culture=neutral, PublicKeyToken=null
e__NativeCall_Assembly, Version=0.0.0.0, Culture=neutral, PublicKeyToken=null
System.Security, Version=4.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a
System.Xml, Version=4.0.0.0, Culture=neutral, PublicKeyToken=b77a5c561934e089
System.Configuration, Version=4.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a
System.Data.SqlXml, Version=4.0.0.0, Culture=neutral, PublicKeyToken=b77a5c561934e089
System.Collections, Version=4.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a
System.Windows, Version=4.0.0.0, Culture=neutral, PublicK

## Using COM Components

Using Microsoft-provided tools such as `aximp.exe` and `tlbimp.exe`,
it is possible to generate managed wrappers for COM libraries. After
generating such a wrapper, you can use the libraries from Python just
like any other managed code.

Note: currently you need to put the generated wrappers in the GAC, in
the `Python.NET` assembly directory or on the PYTHONPATH in order to load them.

## Type Conversion

Type conversion under Python for .NET is fairly straightforward - most
elemental Python types (string, int, long, etc.) convert automatically to
compatible managed equivalents (String, Int32, etc.) and vice-versa.
Note that all strings returned from the CLR are returned as unicode.

Types that do not have a logical equivalent in Python are exposed as
instances of managed classes or structs (System.Decimal is an example).

The .NET architecture makes a distinction between `value types` and
`reference types`. Reference types are allocated on the heap, and value
types are allocated either on the stack or in-line within an object.

A process called `boxing` is used in .NET to allow code to treat a value
type as if it were a reference type. Boxing causes a separate copy of the
value type object to be created on the heap, which then has reference
type semantics.

Understanding boxing and the distinction between value types and reference
types can be important when using Python for .NET because the Python
language has no value type semantics or syntax - in Python
"everything is a reference".

Here is a simple example that demonstrates an issue. If you are an
experienced C# programmer, you might write the following code:

In [97]:
items = System.Array.CreateInstance(Point, 3)
for i in range(3):
    items[i] = Point(0, 0)

items[0].X = 1 # won't work!!

items[0].X

0

While the spelling of `items[0].X = 1` is the same in C# and Python,
there is an important and subtle semantic difference.
In C# (and other compiled-to-IL languages), the compiler knows that
Point is a value type and can do the Right Thing here, changing the
value in place.

In Python however, "everything's a reference", and there is really no
spelling or semantic to allow it to do the right thing dynamically.
The specific reason that `items[0]` itself doesn't change is that when
you say `items[0]`, that getitem operation creates a Python object that
holds a reference to the object at `items[0]` via a GCHandle.
That causes a ValueType (like Point) to be boxed, so the following
setattr (`.X = 1`) _changes the state of the boxed value,
not the original unboxed value_.

The rule in Python is essentially:

> the result of any attribute or item access is a boxed value

and that can be important in how you approach your code.

Because there are no value type semantics or syntax in Python,
you may need to modify your approach. To revisit the previous example,
we can ensure that the changes we want to make to an array item
aren't "lost" by resetting an array member after making changes to it:

In [99]:
items = System.Array.CreateInstance(Point, 3)
for i in range(3):
    items[i] = Point(0, 0)

# This _will_ work. We get 'item' as a boxed copy of the Point
# object actually stored in the array. After making our changes
# we re-set the array item to update the bits in the array.

item = items[0]
item.X = 1
items[0] = item
print(item, items[0], item.X, items[0].X)

{X=1,Y=0} {X=1,Y=0} 1 1


This is not unlike some of the cases you can find in C# where you have to
know about boxing behavior to avoid similar kinds of `lost update` problems
(generally because an implicit boxing happened that was not taken
into account in the code).

This is the same thing, just the manifestation is a little different
in Python. See the .NET documentation for more details on boxing and the
differences between value types and reference types.

## Embedding Python

**Note:** because Python code running under Python for .NET is inherently
unverifiable, it runs totally under the radar of the security infrastructure
of the CLR so you should restrict use of the Python assembly to trusted code.

The Python runtime assembly defines a number of public classes that
provide a subset of the functionality provided by the Python C-API.

These classes include PyObject, PyList, PyDict, PyTuple, etc.
You can review the nPython.exe source code in in "Console.csproj" project
for example of embedding CPython in console .NET app. Please refer to
this README GitHub page for new simplified embedding API:

[README.md][8]

At a very high level, to embed Python in your application you will need to:

-   Reference Python.Runtime.dll in your build environment
-   Call PythonEngine.Intialize() to initialize Python
-   Call PythonEngine.ImportModule(name) to import a module

The module you import can either start working with your managed app
environment at the time its imported, or you can explicitly lookup and
call objects in a module you import.

For general-purpose information on embedding Python in applications,
use www.python.org or Google to find (C) examples. Because
Python for .NET is so closely integrated with the managed environment,
you will generally be better off importing a module and deferring to
Python code as early as possible rather than writing a lot of managed
embedding code.

**Important Note for embedders:** Python is not free-threaded and
uses a global interpreter lock to allow multi-threaded applications
to interact safely with the Python interpreter.
Much more information about this is available in the Python C-API
documentation on the www.python.org Website.

When embedding Python in a managed application, you have to manage the
GIL in just the same way you would when embedding Python in
a C or C++ application.

Before interacting with any of the objects or APIs provided by the
Python.Runtime namespace, calling code must have acquired the Python
global interpreter lock by calling the `PythonEngine.AcquireLock` method.
The only exception to this rule is the `PythonEngine.Initialize` method,
which may be called at startup without having acquired the GIL.

When finished using Python APIs, managed code must call a corresponding
`PythonEngine.ReleaseLock` to release the GIL and allow other threads
to use Python.

The AcquireLock and ReleaseLock methods are thin wrappers over the
unmanaged `PyGILState_Ensure` and `PyGILState_Release` functions from
the Python API, and the documentation for those APIs applies to
the managed versions.

[8]: https://github.com/pythonnet/pythonnet/blob/master/README.md
